In [2]:
import zipfile
import os

# Path file dataset
#dataset_path = "./archive (9).zip"
extract_path = "./dataset"

# Ekstrak file ZIP
#with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
#    zip_ref.extractall(extract_path)

# List file dan folder dalam dataset
os.listdir(extract_path)


['valid_annotations.coco.json',
 'Dental X_Ray',
 'train_annotations.coco.json',
 'test_annotations.coco.json']

File anotasi COCO memiliki struktur standar dengan kunci utama:

info: Informasi dataset.
licenses: Lisensi dataset.
categories: Daftar kategori (label) dalam dataset.
images: Metadata gambar (ID, file name, dsb.).
annotations: Informasi anotasi (bounding box, kategori, dsb.).

menampilkan kategori yang tersedia untuk melihat apakah hanya ada "Caries" dan "Non-Caries" atau lebih.

In [3]:
import json

# Periksa isi file anotasi COCO (contoh dari train_annotations)
annotation_path = os.path.join(extract_path, "train_annotations.coco.json")

with open(annotation_path, "r") as file:
    coco_annotations = json.load(file)

# Tampilkan kunci utama dalam anotasi
coco_annotations.keys()


dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])

Dataset ini memiliki beberapa kategori, termasuk "Caries" (id: 1), serta kategori lain seperti Crown, Filling, Implant, dan sebagainya.
kita fokuskan hanya pada prediksi Caries vs. Non-Caries, kita bisa mengubahnya menjadi task klasifikasi biner:

Caries (1) → Positif
Selain Caries → Negatif

In [4]:
# Menampilkan kategori dalam dataset
coco_annotations["categories"]


[{'id': 0, 'name': 'vzrad2', 'supercategory': 'none'},
 {'id': 1, 'name': 'Caries', 'supercategory': 'vzrad2'},
 {'id': 2, 'name': 'Crown', 'supercategory': 'vzrad2'},
 {'id': 3, 'name': 'Filling', 'supercategory': 'vzrad2'},
 {'id': 4, 'name': 'Implant', 'supercategory': 'vzrad2'},
 {'id': 5, 'name': 'Malaligned', 'supercategory': 'vzrad2'},
 {'id': 6, 'name': 'Mandibular Canal', 'supercategory': 'vzrad2'},
 {'id': 7, 'name': 'Missing teeth', 'supercategory': 'vzrad2'},
 {'id': 8, 'name': 'Periapical lesion', 'supercategory': 'vzrad2'},
 {'id': 9, 'name': 'Retained root', 'supercategory': 'vzrad2'},
 {'id': 10, 'name': 'Root Canal Treatment', 'supercategory': 'vzrad2'},
 {'id': 11, 'name': 'Root Piece', 'supercategory': 'vzrad2'},
 {'id': 12, 'name': 'croen', 'supercategory': 'vzrad2'},
 {'id': 13, 'name': 'impacted tooth', 'supercategory': 'vzrad2'},
 {'id': 14, 'name': 'maxillary sinus', 'supercategory': 'vzrad2'}]

dua pendekatan:

Klasifikasi Biner (Caries vs Non-Caries) → Model CNN sederhana untuk mengklasifikasikan gambar.
Deteksi Objek (Faster R-CNN) → Model Faster R-CNN untuk mendeteksi area karies pada X-ray gigi.

Kita mulai dengan pendekatan klasifikasi terlebih dahulu:

Menggunakan dataset COCO, kita akan mengonversi gambar ke format yang sesuai untuk CNN.
Menandai gambar dengan kategori Caries sebagai positif dan lainnya sebagai negatif.
Melatih model CNN untuk klasifikasi.

In [5]:
import shutil
from PIL import Image

# Buat folder untuk klasifikasi
classification_path = "./dataset/classification_dataset"
os.makedirs(classification_path, exist_ok=True)

caries_path = os.path.join(classification_path, "Caries")
non_caries_path = os.path.join(classification_path, "Non-Caries")
os.makedirs(caries_path, exist_ok=True)
os.makedirs(non_caries_path, exist_ok=True)

# Ambil informasi gambar dan anotasi
image_info = {img["id"]: img["file_name"] for img in coco_annotations["images"]}
annotations = coco_annotations["annotations"]

# Tandai gambar yang mengandung Caries
caries_images = set()
for ann in annotations:
    if ann["category_id"] == 1:  # Caries
        caries_images.add(ann["image_id"])

# Pindahkan gambar sesuai kategori
image_folder = os.path.join(extract_path, "Dental X_Ray")

for img_id, file_name in image_info.items():
    src = os.path.join(image_folder, file_name)
    if not os.path.exists(src):  # Cek apakah file ada
        continue
    
    # Klasifikasi sebagai Caries atau Non-Caries
    if img_id in caries_images:
        dst = os.path.join(caries_path, file_name)
    else:
        dst = os.path.join(non_caries_path, file_name)

    shutil.copy(src, dst)

# Cek jumlah gambar di masing-masing kategori
len(os.listdir(caries_path)), len(os.listdir(non_caries_path))


(0, 0)

Folder "Dental X_Ray" ternyata berisi subfolder "train", "valid", dan "test". Ini berarti gambar tidak langsung berada di dalamnya. Kita akan mencari di dalam subfolder untuk menemukan gambar yang sesuai.

In [6]:
# Periksa beberapa nama file dalam folder "Dental X_Ray"
sample_files = os.listdir(image_folder)[:10]
sample_files


['valid', 'train', 'test']

Gambar berada dalam subfolder "train", "valid", dan "test" dengan format nama file yang panjang. Kita akan menyesuaikan proses pemindahan gambar agar mencakup subfolder ini

In [7]:
# Periksa isi dalam subfolder "train"
train_folder = os.path.join(image_folder, "train")
valid_folder = os.path.join(image_folder, "valid")
test_folder = os.path.join(image_folder, "test")

# Cek beberapa file dalam folder train
os.listdir(train_folder)[:10]


['e7267c04-Faraj_Fatemeh_2022-06-12142338_jpg.rf.a80e9e7b60c4923e1150ab47808b526d.jpg',
 'bee3dc99-FARMANI_AKBAR_2020-08-09114738_jpg.rf.341412c86e8366009658174a00344f9d.jpg',
 '491980000-jpg_png_jpg.rf.9af42221abb2993d39f4115fb0bd7a6f.jpg',
 'cropped_RITISH-GARG_2023-10-20154806_1_png.rf.01f16db955f479e4bf45181d3a055ad6.jpg',
 '0a4f2d22-Hematian_Fariba_57y_31052021_132542_jpg.rf.6bfcf8cfb273a9a0767ea25499da98bc.jpg',
 '4053550000-jpg_png_jpg.rf.35f4b73f5b3cd66f3f3ef281e8d21dec.jpg',
 '4124640000-jpg_png_jpg.rf.5723bb4631f0bc2009f43d310738a33d.jpg',
 '4024990000-jpg_png_jpg.rf.97e746dc542b75d76f9fcd3570205cae.jpg',
 '1098260000-jpg_png_jpg.rf.3c4693abc6422cf1def85166970f2d46.jpg',
 '4066150000-jpg_png_jpg.rf.63fbaf51904f444764269a3fc757095b.jpg']

Sekarang, gambar telah berhasil dipindahkan:

Caries: 1,276 gambar

Non-Caries: 3,496 gambar


Langkah selanjutnya adalah melatih model CNN untuk klasifikasi menggunakan dataset ini. Kita akan menyiapkan model CNN sederhana.

In [8]:
# Fungsi untuk mencari file gambar dalam semua subfolder
def find_image_path(image_name):
    for subfolder in ["train", "valid", "test"]:
        img_path = os.path.join(image_folder, subfolder, image_name)
        if os.path.exists(img_path):
            return img_path
    return None

# Pindahkan gambar ke folder klasifikasi yang sesuai
for img_id, file_name in image_info.items():
    src = find_image_path(file_name)
    if src is None:  # Jika gambar tidak ditemukan, lewati
        continue
    
    # Klasifikasi sebagai Caries atau Non-Caries
    dst_folder = caries_path if img_id in caries_images else non_caries_path
    dst = os.path.join(dst_folder, file_name)

    shutil.copy(src, dst)

# Periksa kembali jumlah gambar yang berhasil dipindahkan
len(os.listdir(caries_path)), len(os.listdir(non_caries_path))


(1276, 3496)

Kita siapkan model CNN untuk klasifikasi karies. Model ini menggunakan beberapa lapisan konvolusi dengan pooling dan dropout untuk menghindari overfitting.

Selanjutnya, kita bisa:

Melatih model dan mengevaluasi performanya.
Menyimpan model untuk prediksi di masa depan.
Melanjutkan ke Faster R-CNN untuk deteksi objek karies.

Berikut untuk melatih model CNN untuk klasifikasi karies di lokal:

Mengatur dataset dari folder

Melatih model CNN dengan TensorFlow

Menyimpan model setelah pelatihan selesai


In [ ]:
#!pip install tensorflow
#jalankan jika belum terpasang tensorflow


Caries Classfification

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

# Parameter model
img_size = (224, 224)
batch_size = 32

dataset_path = "./dataset/classification_dataset"  # Sesuaikan path jika perlu

# Load dataset sebagai image dataset
train_ds = keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

# Model CNN sederhana
model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

# Simpan model setelah pelatihan
model.save("caries_classification_model.h5")


Found 4772 files belonging to 2 classes.
Using 3818 files for training.


W0000 00:00:1741449573.117103   17900 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Found 4772 files belonging to 2 classes.
Using 954 files for validation.
Epoch 1/10


/home/mdsme/Documents/project/Gilang/maskXray/.venv/lib/python3.10/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-03-08 22:59:33.514775: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44302336 exceeds 10% of free system memory.
2025-03-08 22:59:33.533163: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44302336 exceeds 10% of free system memory.
2025-03-08 22:59:33.550416: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44302336 exceeds 10% of free system memory.
2025-03-08 22:59:34.611865: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44302336 exceeds 10% of free system memory.
2025-03-08 22:59:35.497979: W external/local_xla/x

120/120 ━━━━━━━━━━━━━━━━━━━━ 207s 2s/step - accuracy: 0.7815 - loss: 0.5482 - val_accuracy: 0.8438 - val_loss: 0.3593
Epoch 2/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 265s 2s/step - accuracy: 0.8414 - loss: 0.3880 - val_accuracy: 0.8564 - val_loss: 0.3585
Epoch 3/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 0.8517 - loss: 0.3675 - val_accuracy: 0.8700 - val_loss: 0.3266
Epoch 4/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 213s 2s/step - accuracy: 0.8655 - loss: 0.3238 - val_accuracy: 0.8564 - val_loss: 0.3391
Epoch 5/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - accuracy: 0.8868 - loss: 0.2695 - val_accuracy: 0.8899 - val_loss: 0.3295
Epoch 6/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 217s 2s/step - accuracy: 0.9136 - loss: 0.2169 - val_accuracy: 0.8973 - val_loss: 0.3012
Epoch 7/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 222s 2s/step - accuracy: 0.9360 - loss: 0.1566 - val_accuracy: 0.9130 - val_loss: 0.3405
Epoch 8/10
120/120 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - accuracy: 0.9531 - loss: 0.1313 - val_accuracy: 0.939

Setelah pelatihan selesai, model akan disimpan sebagai "caries_classification_model.h5", yang bisa digunakan untuk prediksi di masa depan.



Faster R-CNN digunakan untuk deteksi objek, dalam hal ini mendeteksi lokasi karies dalam gambar X-ray. Kita akan menyiapkan skrip yang mencakup:

Konversi dataset COCO ke format TensorFlow

Melatih Faster R-CNN menggunakan TensorFlow Object Detection API

Menyimpan model hasil pelatihan

Agar bisa berjalan di lokal, pastikan TensorFlow Object Detection API terinstal:

In [ ]:
#!pip install tensorflow tensorflow-gpu opencv-python-headless
#pip install tensorflow-object-detection-api

Faster Rcnn series

In [21]:
import os
import json

# Path ke direktori dataset
dataset_dir = "./dataset"

# File JSON COCO yang ingin dikonversi
annotation_files = {
    "train": os.path.join(dataset_dir, "train_annotations.coco.json"),
    "valid": os.path.join(dataset_dir, "valid_annotations.coco.json"),
    "test": os.path.join(dataset_dir, "test_annotations.coco.json"),
}

# Buat folder labels/ untuk YOLO jika belum ada
for split in ["train", "valid", "test"]:
    os.makedirs(os.path.join(dataset_dir, f"labels/{split}"), exist_ok=True)

# Fungsi konversi COCO JSON ke YOLO TXT
def convert_coco_to_yolo(json_path, output_label_dir):
    with open(json_path, "r") as f:
        coco_data = json.load(f)

    # Mapping kategori COCO ke YOLO (COCO ID -> YOLO ID)
    category_id_map = {cat["id"]: idx for idx, cat in enumerate(coco_data["categories"])}

    # Proses setiap gambar dalam dataset
    for img in coco_data["images"]:
        img_id = img["id"]
        img_width = img["width"]
        img_height = img["height"]
        
        # Buat file label dalam format YOLO
        label_file_path = os.path.join(output_label_dir, f"{img['file_name'].split('.')[0]}.txt")

        with open(label_file_path, "w") as label_file:
            for ann in coco_data["annotations"]:
                if ann["image_id"] == img_id:
                    category_id = category_id_map[ann["category_id"]]
                    x, y, w, h = ann["bbox"]

                    # Konversi ke format YOLO (normalisasi)
                    x_center = (x + w / 2) / img_width
                    y_center = (y + h / 2) / img_height
                    w /= img_width
                    h /= img_height

                    label_file.write(f"{category_id} {x_center} {y_center} {w} {h}\n")

    print(f"✅ Konversi selesai untuk {json_path}")

# Konversi semua split dataset
for split, json_path in annotation_files.items():
    convert_coco_to_yolo(json_path, os.path.join(dataset_dir, f"labels/{split}"))


✅ Konversi selesai untuk ./dataset/train_annotations.coco.json
✅ Konversi selesai untuk ./dataset/valid_annotations.coco.json
✅ Konversi selesai untuk ./dataset/test_annotations.coco.json


In [22]:
yaml_content = """path: ./dataset/Dental X_Ray

train: ./dataset/Dental X_Ray/train  # Folder yang berisi semua gambar
val: ./dataset/Dental X_Ray/valid    # Gunakan gambar yang sama untuk validasi
test: ./dataset/Dental X_Ray/test   # (Opsional) Gunakan gambar yang sama untuk testing

names:
  0: caries
  1: non-caries

"""

with open("./dataset/Dental X_Ray/caries.yaml", "w") as file:
    file.write(yaml_content)

print("✅ File caries.yaml berhasil dibuat!")


✅ File caries.yaml berhasil dibuat!


In [26]:
import os
import glob

image_folder = "./dataset/Dental X_Ray/valid"
label_folder = "./dataset/Dental X_Ray/labels/valid"

images = set(os.listdir(image_folder))
labels = set(os.listdir(label_folder))

missing_labels = images - {f.replace('.jpg', '.txt') for f in images}

print(f"Total gambar: {len(images)}")
print(f"Total anotasi: {len(labels)}")
print(f"Gambar tanpa anotasi: {missing_labels}")


Total gambar: 2072
Total anotasi: 2071
Gambar tanpa anotasi: {'cropped_Ananya-_2023-10-26181943_1_png.rf.2d59023ff41880b2ed94c7b805cb8061.jpg', '3676500000-jpg_png_jpg.rf.a1fdf4a27b2f649a14534ab4ff294323.jpg', '4026390000-jpg_png_jpg.rf.2df893e934846fa9fdb5df650504f780.jpg', '4095150000-jpg_png_jpg.rf.c13809ff542d2474e50a1539d53a132c.jpg', '1682430000-jpg_png_jpg.rf.79b461e774f637702a6940fe35d39ee9.jpg', '3927840000-jpg_png_jpg.rf.44ae4a8fe39b364ec8e9068be3ae80d8.jpg', '7b962e01-MARATI_SEDIGHE_2020-06-08192816_jpg.rf.7f84bb4aad1eb38ef237942b74a8deb7.jpg', '4087150000-jpg_png_jpg.rf.8a7c002234e447dfeb3745465391cf63.jpg', 'ab21c023-KAMALI_FATEMEH_2020-07-19120333_jpg.rf.e7cd165dc9f097e2a52d42a6e7c568b1.jpg', '3930300000-jpg_png_jpg.rf.9cbc4a2c09e9106b54ae49ec17d2058e.jpg', '4110340000-jpg_png_jpg.rf.33feae4b4d1006fcb51ae9c71b6ff8de.jpg', '3825210000-jpg_png_jpg.rf.021e141902f3b9cc86fd2e2d95cb6d0f.jpg', '3891790000-jpg_png_jpg.rf.f028f4a67db51eb86c9b4612e1c94762.jpg', '3888160000-jpg_png_

In [25]:
import os

image_folder = "./dataset/Dental X_Ray/valid"
label_folder = "./dataset/Dental X_Ray/labels/valid"

image_files = {f.replace(".jpg", ".txt") for f in os.listdir(image_folder) if f.endswith(".jpg")}
label_files = {f for f in os.listdir(label_folder) if f.endswith(".txt")}

missing_annotations = image_files - label_files

print("Gambar tanpa anotasi:", missing_annotations)
print(f"Total gambar tanpa anotasi: {len(missing_annotations)}")

for img in missing_annotations:
    label_path = os.path.join(label_folder, img)
    open(label_path, 'w').close()  # Buat file kosong
    print(f"Anotasi kosong dibuat: {label_path}")


Gambar tanpa anotasi: {'1266220000-jpg_png_jpg.rf.12619de38987de9d7afd4582bc720b55.txt', '4033830000-jpg_png_jpg.rf.a5d6bf33ee5eda9cdbb15e089a5062c8.txt', '4175130000_jpg.rf.3ad70baf9eb1645ea8583747f307fa7b.txt', '3766500000-jpg_png_jpg.rf.943913f7c98822154ae7d009a7cd9834.txt', '663710000-jpg_png_jpg.rf.84af8addef40b7f2a9cae3bf2555a7d1.txt', '3997240000-jpg_png_jpg.rf.488475a74c7284d948bec5f9debfb055.txt', '4000300000-jpg_png_jpg.rf.81ef062bb7379f48f52d3fcac5557d87.txt', '4450c195-Rahimi_Moslem_2022-06-12141630_jpg.rf.9aaa17498d6c6ca5df7eb1700a1dfb3b.txt', '3944650000-jpg_png_jpg.rf.2122d7aa6e958f36bf08bdb06d5f1879.txt', '3911450000-jpg_png_jpg.rf.d6d01e5c803c5f3d1506a8fba044c7bd.txt', '3618830000-jpg_png_jpg.rf.3d7ac5aaeb4577ad37ded7ecf2e97a24.txt', '4026450000-jpg_png_jpg.rf.77ba91d0bc7f8265de934aaadacd0d9f.txt', '4161070000_jpg.rf.b04f251efff167d4e773a09c6edb711c.txt', 'cropped_SANTOSH-DEVI_2023-10-26164025_1_png.rf.ee01a3c981594dbcb3020f8b1a23e695.txt', '1540100000-jpg_png_jpg.rf.6

In [37]:
import os
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from IPython.display import display
from PIL import Image

In [38]:
# Cek apakah GPU tersedia
device = "cuda" if torch.cuda.is_available() else "cpu"

# Inisialisasi model YOLO (gunakan pretrained model)
model = YOLO("yolov8n.pt").to(device)  # Gunakan model YOLO pre-trained

In [42]:
# Path dataset COCO (harus sesuai format YOLO)
dataset_dir = "/home/mdsme/Documents/project/Gilang/maskXray/dataset/Dental X_Ray/caries.yaml"

In [44]:

# Fine-tune model dengan dataset COCO
model.train(data=dataset_dir, epochs=10, imgsz=640)


# Simpan model yang sudah dilatih
model.save("/home/mdsme/Documents/project/Gilang/maskXray/dataset/Dental X_Ray/yolo_caries.pt")


engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/mdsme/Documents/project/Gilang/maskXray/dataset/Dental X_Ray/caries.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, for

RuntimeError: Dataset '/home/mdsme/Documents/project/Gilang/maskXray/dataset/Dental X_Ray/caries.yaml' error ❌ 
Dataset '/home/mdsme/Documents/project/Gilang/maskXray/dataset/Dental X_Ray/caries.yaml' images not found ⚠️, missing path '/home/mdsme/Documents/project/Gilang/YoloDental/yolov5/datasets/dataset/Dental X_Ray/dataset/Dental X_Ray/valid'
Note dataset download directory is '/home/mdsme/Documents/project/Gilang/YoloDental/yolov5/datasets'. You can update this in '/home/mdsme/.config/Ultralytics/settings.json'

In [ ]:
# Install TensorFlow 1.x alongside TensorFlow 2.x.
#pip install tensorflow==1.15.5
#pip install tensorflow

!pip install tensorflow==1.15.5
!pip install tf_slim  
!pip install 'protobuf<=3.20.3' #protobuf==3.20.3 works as well

# Restart the runtime after the installation

In [10]:
import os
import tensorflow as tf
from object_detection.utils import config_util
#import object_detection.utils.config_util as config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection.builders import model_builder
import numpy as np
import cv2

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:

# Path dataset dan model
model_dir = "faster_rcnn_model"
dataset_dir = "./dataset/classification_dataset"  # Sesuaikan dengan lokasi dataset COCO
pipeline_config = os.path.join(model_dir, "pipeline.config")

# Load konfigurasi model
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']

detection_model = model_builder.build(model_config=model_config, is_training=False)

# Load checkpoint terakhir
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt_manager = tf.train.CheckpointManager(ckpt, model_dir, max_to_keep=5)
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint).expect_partial()
    print("Model restored from checkpoint.")

# Fungsi training step
def train_step(image_tensors, groundtruth_boxes, groundtruth_classes):
    with tf.GradientTape() as tape:
        detections = detection_model(
            image_tensors,
            groundtruth_boxes=groundtruth_boxes,
            groundtruth_classes=groundtruth_classes,
            training=True
        )
        total_loss = detections["loss"]
    gradients = tape.gradient(total_loss, detection_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, detection_model.trainable_variables))
    return total_loss

# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# Loop training
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    for batch in dataset_loader(dataset_dir):  # Implementasikan dataset_loader sesuai dataset COCO
        images, boxes, classes = batch
        loss = train_step(images, boxes, classes)
        print("Loss:", loss.numpy())

# Simpan model
ckpt_manager.save()
print("Faster R-CNN model saved.")

# Evaluasi Model dengan Visualisasi
def evaluate_model(image_path):
    image = cv2.imread(image_path)
    image_resized = cv2.resize(image, (224, 224))
    image_tensor = tf.convert_to_tensor(image_resized, dtype=tf.float32)
    image_tensor = tf.expand_dims(image_tensor, 0)
    
    detections = detection_model(image_tensor, training=False)
    
    # Ambil bounding box dan skor prediksi
    boxes = detections['detection_boxes'][0].numpy()
    scores = detections['detection_scores'][0].numpy()
    threshold = 0.5  # Ambang batas untuk menampilkan prediksi
    
    for i in range(len(boxes)):
        if scores[i] > threshold:
            h, w, _ = image.shape
            ymin, xmin, ymax, xmax = boxes[i]
            xmin, xmax, ymin, ymax = int(xmin * w), int(xmax * w), int(ymin * h), int(ymax * h)
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            label = f"Caries: {scores[i]:.2f}"
            cv2.putText(image, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    cv2.imshow("Detections", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Contoh evaluasi
sample_image = "path/to/sample_image.jpg"  # Ubah dengan gambar tes
evaluate_model(sample_image)


Pastikan dataset COCO tersedia di dataset_coco.

Modifikasi fungsi dataset_loader agar memuat data dengan format yang sesuai.

Jalankan skrip untuk melatih model.

🔹 Cara Menggunakan Evaluasi:

Ubah sample_image menjadi path gambar yang ingin diuji.

Jalankan evaluate_model(sample_image).

Model akan menampilkan hasil deteksi.


Kita telah menambahkan visualisasi bounding box ke dalam evaluasi model.

Bounding box hijau akan muncul pada area yang terdeteksi sebagai karies.

Skor prediksi akan ditampilkan di atas bounding box.

Tekan tombol apa saja untuk menutup gambar setelah evaluasi.

Skrip diatas melatih model Faster R-CNN untuk mendeteksi karies dalam X-ray gigi.
Setelah pelatihan selesai, model akan disimpan di direktori faster_rcnn_model